In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import BeamDynamics as bd
import SimulationData as sd
import RFTrackTools as rfttools
import copy
import json

In [ ]:
from importlib import reload
reload(bd)
reload(rfttools)

In [ ]:
# %matplotlib inline
# %matplotlib notebook
%matplotlib widget
plt.rcParams['figure.figsize'] = [9.6, 6.4]
defaultColorCycle = plt.rcParams["axes.prop_cycle"].by_key()['color']
# plotFont = {
#     'family' : 'sans-serif',
#     'weight' : 'normal',
#     'size'   : 12
# }
# matplotlib.rc('font', **plotFont)
# plt.rc('legend', fontsize=10)

# Transverse Emittance Analisys

## Distribution at Target Exit

In [ ]:
DISTR_REL_PATH = '../../Data/Geant4/Pcubed/P3dist_baseline_p.ini'
targetDistr, _ = bd.convert_astra_to_standard_df(DISTR_REL_PATH, delim_whitespace=False, filterSpecsSelector=None, zProjection=None)
targetDistr.describe()

In [ ]:
plotSets = ['TransvPlane', 'TransvPsAngles', 'TransvPsMomenta', 'LongPsT']
plotDefs = bd.set_plot_defs_from_distrs([targetDistr], setNames=plotSets)
plotDefs[6]['lims2'] = [0., 100.]
_ = bd.plot_distr([targetDistr], plotDefs)

## Overall Emittance

In [ ]:
emitn = {}
emitGeom = {}
emitTraceSpace = {}
alphaTwiss = {}
betaTwiss = {}
gammaTwiss = {}
for planeName in ('x', 'y'):
    emitn[planeName] = bd.compute_emittance(targetDistr, planeName, norm='normalized')
    emitGeom[planeName] = bd.compute_emittance(targetDistr, planeName, norm='geometric')
    emitTraceSpace[planeName] = bd.compute_emittance(targetDistr, planeName, norm='tracespace')
    alphaTwiss[planeName], betaTwiss[planeName], gammaTwiss[planeName] = bd.compute_twiss(targetDistr, planeName)
    print(
        'emitn_{0:s} = {1:.1f} pi mm mrad, emitGeom_{0:s} = {2:.1f} pi mm mrad, emitTraceSpace_{0:s} = {3:.1f} pi mm mrad.'.format(
            planeName, emitn[planeName], emitGeom[planeName], emitTraceSpace[planeName]))
    print(
        'alphaTwiss_{0:s} = {1:.3f}, betaTwiss_{0:s} = {2:.3f} mm, gammaTwiss_{0:s} = {3:.3f} 1/mm.'.format(
            planeName, alphaTwiss[planeName], betaTwiss[planeName], gammaTwiss[planeName]))

In [ ]:
emitRef = emitTraceSpace['x']

## Transverse Emittance Vs. Bunch Portion

### Values vs. 1D (?) Gaussian Distribution (To Be Revised)

In [ ]:
gaussianPortions = {1: 0.6827, 2: 0.9545, 3: 0.9973, 4: 0.999937, 6: 0.99999998}
ellipseSpecs = {
    'x': {'alphaTwiss': alphaTwiss['x'], 'betaTwiss': betaTwiss['x']},
    'y': {'alphaTwiss': alphaTwiss['y'], 'betaTwiss': betaTwiss['y']}}
for planes in (['x'], ['x', 'y']):
    print('\nFiltering on {:d} planes:'.format(len(planes)))
    for Fa in (1., 2., 3., 4., 6.):
        selEllipseSpecs = {k: v for k, v in ellipseSpecs.items() if k in planes}
        distrWithinFaSigma, portionWithinFaSigma = bd.distr_within_ellipse(targetDistr, Fa**2.*emitRef, selEllipseSpecs)
        print(
            'Portion within {:.1f} sigma: {:.3f} (vs. {:.3f} for Gaussian).'.format(Fa, portionWithinFaSigma, gaussianPortions[Fa]))

In [ ]:
refEllipseSpecs = {
    'x': {'alphaTwiss': alphaTwiss['x'], 'betaTwiss': betaTwiss['x']},
    'y': {'alphaTwiss': alphaTwiss['y'], 'betaTwiss': betaTwiss['y']}
}
FaList = np.arange(6., 0, -0.2)
distrWithinRef = []
bunchPortions = []
emitnPortions = []
for Fa in FaList:
    distrWithinRef.append(bd.distr_within_ellipse(targetDistr, Fa**2.*emitRef, refEllipseSpecs)[0])
    bunchPortions.append(distrWithinRef[-1].shape[0]/targetDistr.shape[0])
    emitnPortions.append(bd.compute_emittance(distrWithinRef[-1], 'x', verbose=False))

### Emittance Vs. Bunch Portion

In [ ]:
indsToAnnotate = [-5, -10, -15, -20, -25]
fig, ax = plt.subplots()
ax.plot(bunchPortions, emitnPortions, 'o-')
for ind in indsToAnnotate:
    ax.annotate(np.round(FaList[ind], 1), (bunchPortions[ind]-0.05, emitnPortions[ind]))
ax.set_xlim([0, 1])
ax.set_ylim([0, 14e3])
ax.set_xlabel('Bunch portion')
ax.set_ylabel('Norm. transv. emittance [mm mrad]')
ax.grid()

#### Export for Nico

In [ ]:
# xy = np.column_stack([bunchPortions, emitnPortions])
# np.savetxt('emitnVsBunchPortion.dat', xy, header='{:24s} {:24s}'.format('bunchPortion', 'emitn [mm mrad]'))

### Bunch Portion Vs. Cut Range

In [ ]:
fig, ax = plt.subplots()
ax.plot(FaList, bunchPortions, 'o-')
ax.set_xlim([0, np.max(FaList)])
ax.set_ylim([0, np.max(bunchPortions)])
ax.set_xlabel('Fa of cut with Fa^2*rmsEmit ellipse')
ax.set_ylabel('Bunch portion')
ax.grid()

### Visualization of Filtered Distributions

In [ ]:
stepsToPlot = np.arange(5*2, len(distrWithinRef), 2)
FaListToPlot = [FaList[stepInd] for stepInd in stepsToPlot]
plotDefs[1]['lims1'] = [-15., 15.]
plotDefs[2]['lims1'] = plotDefs[1]['lims1']
plotDefs[1]['displayTable'] = False
ax = bd.plot_distr(
    [targetDistr, *[distrWithinRef[stepInd] for stepInd in stepsToPlot]], plotDefs,
    legendLabels=[
        'Full bunch',
        *['Within {:.1f} sigma'.format(Fa) for Fa in FaListToPlot]])
for ind, Fa in enumerate(FaListToPlot):
    bd.plot_ellipse(
        ax[1][0,0], Fa**2.*emitRef, semiAxisOrder=1, alphaTwiss=alphaTwiss['x'], betaTwiss=betaTwiss['x'], color='k')
    bd.plot_ellipse(
        ax[2][0,0], Fa**2.*emitRef, semiAxisOrder=1, alphaTwiss=alphaTwiss['y'], betaTwiss=betaTwiss['y'], color='k')

## Transverse Emittance of Different Energy Windows

In [ ]:
EkinWindowEdges = np.arange(0., 500., 2.)
particlesInWindow = []
emitnWindows = []
for wInd in range(len(EkinWindowEdges)-1):
    EkinWindow = {"Ekin": EkinWindowEdges[wInd:wInd+2]}
    particlesInWindow.append(bd.filter_distr(targetDistr, EkinWindow))
    emitnWindows.append(bd.compute_emittance(particlesInWindow[wInd], 'x', verbose=False))

### Emittannce Vs. Energy

In [ ]:
fig, ax = plt.subplots()
ax.plot(EkinWindowEdges[:-1], emitnWindows, 'o-')
ax.set_ylim([0, 15e3])
ax.set_xlabel('Ekin window bottom edge [MeV]')
ax.set_ylabel('Norm. emittance [pimmmrad]')
ax.grid()

### Visualization of Filtered Distributions

In [ ]:
windowIndsToPlot = np.arange(0, len(particlesInWindow), 1)
windowsToPlot = [EkinWindowEdges[wInd:wInd+2] for wInd in windowIndsToPlot]
ax = bd.plot_distr(
    [targetDistr, *[particlesInWindow[wInd] for wInd in windowIndsToPlot]], plotDefs,
    legendLabels=[
        'Full bunch',
        *['{:.1f} MeV < Ekin < {:.1f} MeV'.format(*EkinWindow) for EkinWindow in windowsToPlot]])

## Yield at DR Vs. Energy Window Size

### Distribution at DR

In [ ]:
DISTR_REL_PATH = {}
FILTER_SPECS = {}
REF_PARTICLE_PZ = {}
DISTR_REL_PATH['wp1'] = '../../Data/Astra/WP1atDR.dat'
FILTER_SPECS['wp1'] = {
    'z': [14480., 14530.],
    'pz': [0., 1620.]}
REF_PARTICLE_PZ['wp1'] = 1565.  # [MeV/c]
DISTR_REL_PATH['wp2'] = '../../Data/Astra/WP2atDR.dat'
FILTER_SPECS['wp2'] = {
    'z': [14470., 14520.],
    'pz': [0., 1620.]}
REF_PARTICLE_PZ['wp2'] = 1565.  # [MeV/c]

In [ ]:
drDistr = {}
for wpName, relPath in DISTR_REL_PATH.items():
    drDistr[wpName] = pd.read_csv(relPath, names=bd.FILE_TYPES_SPECS['astra']['columnOrder'], header=None)
    # Minimal substitution of drDistr[wpName] = bd.extend_standard_df(drDistr[wpName], False)
    drDistr[wpName]['xp'] = 0.
    drDistr[wpName]['yp'] = 0.
    drDistr[wpName]['t'] = 0.
    drDistr[wpName]['Ekin'] = 0.
    drDistr[wpName]['pdgId'] = -11
    print('Working point {:s}:'.format(wpName))
    print(drDistr[wpName].describe())

In [ ]:
for wpName, filterSpecs in zip(drDistr.keys(), FILTER_SPECS.values()):
    drDistr[wpName] = bd.filter_distr(drDistr[wpName], filterSpecs)
    print('Working point {:s}:'.format(wpName))
    print(drDistr[wpName].describe())

In [ ]:
plotSets = ['z-pz']
plotDefs = bd.set_plot_defs_from_distrs(drDistr.values(), setNames=plotSets)
plotDefs[0]['lims1'] = [14488., 14516.]
plotDefs[0]['lims2'] = [1420., 1620.]
for distr in drDistr.values():
    _ = bd.plot_distr([distr], plotDefs)

### DR Acceptance Vs. Energy Window Amplitude

In [ ]:
def particles_in_window(distr, refParticlePz, pzWindowHalfAmplitudes, pzWindowTopEdgeScanRange, pzWindowAmplitudeStep):
    particlesInWindow = []
    for pzWindowAmplitude in pzWindowHalfAmplitudes:
        optYield = 0
        PZ_WINDOW_TOP_STEP = 1.  # [MeV/c]
        for pzWindowTop in np.arange(distr['pz'].max(), distr['pz'].max()-pzWindowTopEdgeScanRange*(1+PZ_WINDOW_TOP_STEP), -PZ_WINDOW_TOP_STEP):
            pzWindow = {
                "pz": [pzWindowTop-2.*pzWindowAmplitude*refParticlePz, pzWindowTop]
            }
            tmpParticlesInWindow = bd.filter_distr(distr, pzWindow)
            if tmpParticlesInWindow.shape[0] > optYield:
                optYield = tmpParticlesInWindow.shape[0]
                optParticlesInWindow = tmpParticlesInWindow
        particlesInWindow.append(optParticlesInWindow)
    return particlesInWindow

In [ ]:
REF_WINDOW_HALF_AMPLITUDE = 0.038
MAX_WINDOW_HALF_AMPLITUDE = 0.06
PZ_WINDOW_AMPLITUDE_STEP = 0.001
PZ_WINDOW_TOP_EDGE_SCAN_RANGE = 200.  # [MeV/c]
pzWindowHalfAmplitudes = np.flip(np.arange(PZ_WINDOW_AMPLITUDE_STEP, MAX_WINDOW_HALF_AMPLITUDE+PZ_WINDOW_AMPLITUDE_STEP, PZ_WINDOW_AMPLITUDE_STEP))
particlesInWindow = {}
totParticlesInWindow = {}
for wpName in drDistr.keys():
    particlesInWindow[wpName] = particles_in_window(drDistr[wpName], REF_PARTICLE_PZ[wpName], pzWindowHalfAmplitudes, PZ_WINDOW_TOP_EDGE_SCAN_RANGE, PZ_WINDOW_AMPLITUDE_STEP)
    totParticlesInWindow[wpName] = [partInWin.shape[0] / drDistr[wpName].shape[0] for partInWin in particlesInWindow[wpName]]
    totParticlesInRefWindow = np.interp(REF_WINDOW_HALF_AMPLITUDE, np.flip(pzWindowHalfAmplitudes), np.flip(totParticlesInWindow[wpName]))
    totParticlesInWindow[wpName] /= totParticlesInRefWindow

In [ ]:
fig, ax = plt.subplots()
for wpName in drDistr.keys():
    ax.plot(pzWindowHalfAmplitudes*100., totParticlesInWindow[wpName], '-')
    ax.plot(REF_WINDOW_HALF_AMPLITUDE*100., 1., 'ro')
ax.set_xlim([0., 5.])
ax.set_ylim([0., 1.2])
ax.set_xlabel('pz window amplitude [%]')
ax.set_ylabel('Particles in window')
ax.legend([wpName.upper() for wpName in drDistr.keys() for _ in range(2)])
ax.grid()

#### Export for Nico

In [ ]:
# xy1y2 = np.column_stack([pzWindowHalfAmplitudes*100., *totParticlesInWindow.values()])
# np.savetxt('drAcceptanceVsPzWindowAmplitude.dat', xy1y2, header='{:24s} {:24s} {:24s}'.format('pzWindowHalfAmpl [%]', 'DRAcceptanceWP1', 'DRAcceptanceWP2'))

### Visualization of Filtered Distributions

In [ ]:
stepsToPlot = np.arange(0, len(particlesInWindow['wp1']), 10)
pzWindowHalfAmplitudesToPlot = [pzWindowHalfAmplitudes[stepInd] for stepInd in stepsToPlot]
for wpName in drDistr:
    ax = bd.plot_distr(
        [drDistr[wpName], *[particlesInWindow[wpName][stepInd] for stepInd in stepsToPlot]], plotDefs,
        legendLabels=[
            'Full bunch',
            *['Within +- {:.1f} %'.format(pzWindowHalfAmplitude*100.) for pzWindowHalfAmplitude in pzWindowHalfAmplitudesToPlot]])